<a href="https://colab.research.google.com/github/yaantow/bdds-sentiment_analysis/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q google-genai pandas tqdm

In [ ]:
import os
from google.colab import userdata

 # Load the API key securely from Colab's Secret Vault
api_key = userdata.get("GEMINI_API_KEY")

if not api_key:
  raise ValueError(
     	"No GEMINI_API_KEY found in Colab Secrets.\n"
     	"Go to the left sidebar → 🔑 'Secrets' → Add GEMINI_API_KEY."
 	)

 # Expose it as an environment variable so the Gemini SDK can pick it up
os.environ["GEMINI_API_KEY"] = api_key

print("✅ Gemini API key loaded securely from Colab Secrets.")


✅ Gemini API key loaded securely from Colab Secrets.


In [ ]:
from google import genai
from google.genai import types
import pandas as pd
from tqdm import tqdm

tqdm.pandas() #enables progress bars in pandas

#create the gemini client(uses GEMINI_API_KEY automatically)
client = genai.Client()
print ("✅ Gemini client created")

✅ Gemini client created


# Sentiment Analysis with Gemini API (No Model Training)

 This notebook will:
 - Load a dataset of text entries
 - Send each text to Gemini using an API call
 - Receive a sentiment label: Positive, Negative, or Neutral
 - Prepare groundwork for full‑dataset analysis


In [ ]:
#change this filename to your uploaded csv

csv_path="/content/drive/MyDrive/CYX/BDDS/twitter_sentiment.csv"

df=pd.read_csv(csv_path)
print("Columns:", df.columns.tolist())
df.head()

Columns: ['2401', 'Borderlands', 'Positive', 'im getting on borderlands and i will murder you all ,']


,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


## Expected Sentiment Output

 Gemini will return one of the following:
 - Positive
 - Negative
 - Neutral


In [ ]:
sample_text = "I really loved this product, it works perfectly!"

prompt = f"""
You are a sentiment analysis classifier.

 Task:
 - Read the user's text.
 - Decide if the overall sentiment is Positive, Negative, or Neutral.
 - Return ONLY one word: Positive, Negative, or Neutral.

 Text:
 \"\"\"{sample_text}\"\"\"
 """
response = client.models.generate_content(
  model="gemini-2.5-flash",
  contents=prompt,
)

print("Raw response:", repr(response.text))

Raw response: 'Positive'


In [ ]:
def classify_sentiment(text: str) -> str:
    """
    Use Gemini (gemini-2.5-flash) to classify sentiment of a single text.
    Returns one of: Positive, Negative, Neutral, Unknown, Error.
    """
    # Handle missing or non-string inputs
    if not isinstance(text, str) or text.strip() == "":
        return "Unknown"

    prompt = f"""
You are a sentiment analysis classifier.

Task:
- Read the user's text.
- Decide if the overall sentiment is Positive, Negative, or Neutral.
- Return ONLY one word: Positive, Negative, or Neutral.

Text:
\"\"\"{text}\"\"\"
"""

    try:
        response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=prompt,
            config=types.GenerateContentConfig(
                temperature=0.0,      # deterministic
                max_output_tokens=10, # short answer
            ),
        )

        raw = (response.text or "").strip()
        # Take only the first word to be safe (sometimes models add extra text)
        label = raw.split()[0].capitalize() if raw else "Unknown"

        if label not in {"Positive", "Negative", "Neutral"}:
            return "Unknown"

        return label

    except Exception as e:
        # For debugging; in production you'd log this
        print("❌ Error while calling Gemini:", e)
        return "Error"
